In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import selenium.webdriver.support.ui as ui
import time
chrome_options = Options()  
chrome_options.add_argument("--headless")
br = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'./chromedriver.exe')

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [2]:
# Warning: code takes more than an hour to run since we have to wait until every
# page's Javascript content loads. That takes around 1-2 seconds normally, but
# sometimes may take one second longer than that, causing the program to crash.
# We use 5 seconds just in case, to avoid any possible hangups in the process.

# Scrapes median income, unemployment rate, and mean commute times of every city
def scraping_cities(city, row):
  
    br.get("https://factfinder.census.gov/faces/nav/jsf/pages/community_facts.xhtml")
    wait = ui.WebDriverWait(br,15)
    time.sleep(2)
    # Types search terms for a city
    element = wait.until(lambda br:br.find_element_by_id('cfsearchtextbox'))
    element.clear()
    element.send_keys(city)

    # Submits form
    element =  wait.until(lambda br: br.find_element_by_xpath('//*[@id="communityfactssubmit"]'))
    element.click()

    # Clicking the income tab
    time.sleep(2.5)
    element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="leftnav"]/a[7]'))
    element.click()

    # Getting table link
    time.sleep(2.5)
    element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="cf-content"]/div[2]/div[1]/ul[1]/li[1]/div/a'))
    href = element.get_attribute('href')
    br.get(href)

    #
    for i in range(2, 9):
        time.sleep(5)
            
        # Append unemployment rate
        element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="data"]/tbody/tr[11]/td[3]'))
        unemploymentDF.iloc[row, i-2] = element.text
        
        # Append commute time rate
        element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="data"]/tbody/tr[33]/td[1]'))
        commuteDF.iloc[row, i-2] = element.text
        
        # Append median income data
        element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="data"]/tbody/tr[79]/td[1]'))
        incomeDF.iloc[row, i-2] = element.text
        
        # Stop from getting nonexistent element
        if i == 8:
            break
        
        # Get previous year
        element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="year_selector_content"]/ul/li['+ str(i) +']/a'))
        element.click()


In [3]:
cities = [line.rstrip('\n\r') for line in open('./cities.txt')]

# Setting up the column names for each data frame
incomeDF = pd.DataFrame(columns=["City", "Median Income 2016", "Median Income 2015", "Median Income 2014", "Median Income 2013",
                  "Median Income 2012", "Median Income 2011", "Median Income 2010"])

unemploymentDF = pd.DataFrame(columns=["City", "Unemployment Rate 2016", "Unemployment Rate 2015", "Unemployment Rate 2014", 
                                       "Unemployment Rate 2013", "Unemployment Rate 2012", "Unemployment Rate 2011", 
                                       "Unemployment Rate 2010"])

commuteDF = pd.DataFrame(columns=["City", "Mean Commute Time 2016", "Mean Commute Time 2015", "Mean Commute Time 2014", 
                                  "Mean Commute Time 2013", "Mean Commute Time 2012", "Mean Commute Time 2011", 
                                  "Mean Commute Time 2010"])

# Set the amount of rows per data frame
data = pd.DataFrame({"City": range(len(cities))})
incomeDF = incomeDF.append(data)
unemploymentDF = unemploymentDF.append(data)
commuteDF = commuteDF.append(data)

# Input the name of a city per row in each data frame
for i, row in incomeDF.iterrows():
    row.loc['City'] = cities[i]
    
for i, row in unemploymentDF.iterrows():
    row.loc['City'] = cities[i]
    
for i, row in commuteDF.iterrows():
    row.loc['City'] = cities[i]


# Reverse the column order (ie 2010 to 2016)
incomeDF = incomeDF.iloc[:, ::-1]
unemploymentDF = unemploymentDF.iloc[:, ::-1]
commuteDF = commuteDF.iloc[:, ::-1]

for i, city in enumerate(cities):
    scraping_cities(city, i)
    print(city + " " + str(i)) # Done to see what city we're currently scraping

# Setting the "City" as the index column for all dataframes
incomeDF = incomeDF.set_index('City')
unemploymentDF = unemploymentDF.set_index('City')
commuteDF = commuteDF.set_index('City')

# Close the webdriver instance
br.quit()

FileNotFoundError: [Errno 2] No such file or directory: './cities.txt'